In [1]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv
from typing import List, Dict, Any
import mysql.connector
from mysql.connector import Error

# Load environment variables
dotenv_path = os.path.join('..', 'keys.env')
load_dotenv(dotenv_path)
api_key = os.getenv("OPENAI_API_KEY")
db_user = os.getenv("DATABASE_USER")
db_password = os.getenv("DATABASE_PASSWORD")

def connect_to_mysql():
    connection = mysql.connector.connect(
        host='box5882.bluehost.com',
        database='detoraki_fetusdata',
        user=db_user,
        password=db_password,
        port=3306  # Specify default MySQL port
    )
    return connection

def get_schema() -> Dict[str, List[str]]:
    conn = connect_to_mysql()
    try:
        cursor = conn.cursor()
        
        # Get all tables
        cursor.execute("SHOW TABLES;")
        tables = cursor.fetchall()
        
        schema = {}
        for table in tables:
            table_name = table[0]
            cursor.execute(f"DESCRIBE {table_name};")
            columns = [row[0] for row in cursor.fetchall()]
            schema[table_name] = columns
            
        cursor.close()
        return schema
    except Error as e:
        print(e)
    finally:
        conn.close()

schema = get_schema()

def generate_sql_query(user_query, schema, api_key, conversation_history):
    openai.api_key = api_key
    
    # Prepare the schema information as part of the system message
    schema_prompt = "Here is the database schema:\n"
    for table, columns in schema.items():
        schema_prompt += f"Table: {table}\nColumns: {', '.join(columns)}\n"
    
    # Use the Chat Completion API (for models like GPT-4)
    messages = [
        {
            "role": "system",
            "content": (
                "You are a SQL expert specialized in phpMyAdmin."
                "Answer the questions by providing an SQL script that is compatible with MySQL."
                "Respect the following schema:\n"
                "The lastName AND firstName columns in s_org_ext are used interchangeably.\n"
                + schema_prompt +
                "Ensure that Greek names remain in Greek, without transliteration. "
                "Do not provide any explanations, comments, or additional context—only return the SQL query as plain text.\n"
                "The visits can be found in the combination of tblobstetrichistory_x and tblgynhistory tables since the tblappointment isn't actively used.\n"
                "for the number of children, you need to count the number of rows in tblpregnancyhistory.\n"
                
                "Do not use any other tables than the ones provided in the schema.\n" 
                "Do not use columns that are not in the respective table.\n"
                "s_org_extID is not in tblobstetrichistory_x so you need to join with tblobstetrichistory.\n"
                "The lastName AND firstName columns in s_org_ext are used interchangeably.\n"
                "Ensure the SQL query is syntactically correct and uses only the columns and tables specified in the schema."

            )
        }
    ] + conversation_history + [
        {
            "role": "user",
            "content": f"User query: {user_query}"
        }
    ]
    
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Ensure you're using a chat-based model
        messages=messages,
        max_tokens=250,
        temperature=0,
        top_p=1,
    )
    
    # Extract the SQL query from the assistant's reply
    sql_query = response['choices'][0]['message']['content'].strip()
    return sql_query

def run_query(query_string):
    conn = connect_to_mysql()
    try:
        cursor = conn.cursor()
        cursor.execute(query_string)
        results = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(results, columns=column_names)
        return df
    except Error as e:
        print(e)
        print(query_string)
    finally:
        conn.close()

def synthesize_response(user_query, data, api_key, conversation_history):
    openai.api_key = api_key
    
    # Build a conversation prompt where the data retrieved from the database is included
    data_context = f"Retrieved data: {data}\n"

    # Messages for the chat model (GPT-4) to synthesize the final response
    messages = conversation_history + [
        {
            "role": "system",
            "content": "You are a helpful assistant that provides concise and accurate answers based on user queries and retrieved data."
        },
        {
            "role": "user",
            "content": f"User query: {user_query}"
        },
        {
            "role": "system",
            "content": data_context  # Provide the retrieved data as context for GPT-4
        }
    ]
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        max_tokens=1000,
        temperature=0,
        top_p=1,
    )
    
    final_response = response['choices'][0]['message']['content'].strip()
    return final_response

def rag_pipeline(user_query, api_key=None, conversation_history=[]):
    # Step 1: Get the database schema dynamically
    schema = get_schema()
    
    # Step 2: Generate the SQL query from the user's natural language question
    sql_query = generate_sql_query(user_query, schema, api_key, conversation_history)
    
    # Step 3: Execute the SQL query and retrieve data from the database
    result = run_query(sql_query)
    
    # Step 4: Synthesize the final response using GPT-4
    final_response = synthesize_response(user_query, result, api_key, conversation_history)
    
    return final_response, sql_query, result

# Function to handle the recursive question-answer process
def ask_questions():
    conversation_history = []
    while True:
        user_query = input("Enter your question (or type 'exit' to quit): ")
        print(f"Ερώτηση: {user_query}")
        
        # Exit condition
        if user_query.lower() == 'exit':
            print("Exiting the question-answer loop.")
            break

        # Process the query using your pipeline
        final_response, sql_query, result = rag_pipeline(user_query, api_key=api_key, conversation_history=conversation_history)
        
        # Append the user query and the assistant's response to the conversation history
        conversation_history.append({"role": "user", "content": user_query})
        conversation_history.append({"role": "assistant", "content": final_response})
        
        print(f"Απάντηση: {final_response}") # Print the response for the current query
        print(f"SQL Query: \n{sql_query}")
        print(f"Αποτελέσματα: \n{result}")
        print("--------------------------------------")

In [2]:
ask_questions()

Ερώτηση: How many patients do I have?
Απάντηση: You have 1435 patients.
SQL Query: 
SELECT COUNT(*) FROM s_org_ext WHERE IsActive = 1;
Αποτελέσματα: 
   COUNT(*)
0      1435
--------------------------------------
Ερώτηση: How old is Φανη Βασιλειαδου;
Απάντηση: Φανη Βασιλειαδου is 79 years old.
SQL Query: 
SELECT TIMESTAMPDIFF(YEAR, dateOfBirth, CURDATE()) AS Age 
FROM s_org_ext 
WHERE firstName = 'Φανη' AND lastName = 'Βασιλειαδου';
Αποτελέσματα: 
   Age
0   79
--------------------------------------
Ερώτηση: exit
Exiting the question-answer loop.


In [ ]:
run_query("""

# SELECT *
# FROM tblobstetrichistory 
# # INNER JOIN tblobstetrichistory_x ON tblobstetrichistory.ObstetricHistoryID = tblobstetrichistory_x.ObstetricHistoryID 
# WHERE tblobstetrichistory.s_org_extID = 3;
          
# select * from tblobstetrichistory_x where ObstetricHistoryID = 431;
          
select * from tblpregnancyhistory_x where PregnancyID = 569;
# select * from tblpregnancyhistory where s_org_extID = 820;

"""
          )



,Pregnancy_XID,PregnancyID,DateOfVisit,CauseOfVisit,Tokos,PregnancyAge,Height,WeightBegin,WeightCurrent,WeightChange,...,CervicalEffacement,ArterialPressure,Temperature,SumphysialFundalHeightSFH,Comments,Created,CreatedBy,LastUpdated,LastUpdatedBy,IsActive
0,3943,569,2024-04-15,,0,6+6(wbd),0.0,0.0,0.0,0.0,...,,,0.0,0.0,"-υπέρηχος κυήσεως: ένα έμβρυο, AUA 7+3 weeks\r...",2024-04-15 18:42:00,stelios77,2024-04-15 19:06:57,stelios77,1
1,3946,569,2024-04-30,,0,8+7(wbd),0.0,0.0,0.0,0.0,...,,,0.0,0.0,"-υπέρηχος κυήσεως: ένα έμβρυο, AUA 9+3 weeks, ...",2024-04-15 21:49:36,stelios77,2024-04-30 19:04:39,stelios77,1
2,3970,569,2024-05-14,,0,10+7(wbd),0.0,0.0,0.0,0.0,...,,,0.0,0.0,"-υπέρηχος κυήσεως: ένα έμβρυο, AUA 11+6 weeks\...",2024-04-30 22:03:46,stelios77,2024-06-07 20:53:47,stelios77,1
3,3997,569,2024-06-04,,0,13+7(wbd),0.0,0.0,0.0,0.0,...,,,0.0,0.0,"-υπέρηχος κυήσεως: ΑΡΡΕΝ, όλα καλά\r\n-ΑΠ: 120...",2024-05-14 21:39:59,stelios77,2024-06-04 19:09:52,stelios77,1
4,4046,569,2024-07-02,,0,17+7(wbd),0.0,0.0,0.0,0.0,...,,,0.0,0.0,"-υπέρηχος κυήσεως: ΑΡΡΕΝ, AUA 18+5 weeks, όλα ...",2024-06-04 19:13:08,stelios77,2024-07-02 19:11:12,stelios77,1
5,4088,569,2024-08-06,,0,22+7(wbd),0.0,0.0,0.0,0.0,...,,,0.0,0.0,"-υπέρηχος κυήσεως: ΑΡΡΕΝ, όλα καλά\r\n-ΑΠ: 120...",2024-07-02 21:36:55,stelios77,2024-08-06 19:11:31,stelios77,1
6,4129,569,2024-09-02,,0,26+6(wbd),0.0,0.0,0.0,0.0,...,,,0.0,0.0,"-υπέρηχος κυήσεως: ΑΡΡΕΝ, κ/ι, AUA 27 weeks\r\...",2024-08-06 21:04:19,stelios77,2024-09-02 18:49:34,stelios77,1
7,4168,569,2024-10-01,,0,30+7(wbd),0.0,0.0,0.0,0.0,...,,,0.0,0.0,"-υπέρηχος κυήσεως: ΑΡΡΕΝ, όλα καλά\r\n-ΑΠ:, ΣΒ...",2024-09-02 18:53:34,stelios77,2024-10-01 19:33:53,stelios77,1
8,4201,569,2024-10-31,,0,35+2(wbd),0.0,0.0,0.0,0.0,...,,,0.0,0.0,"-υπέρηχος κυήσεως: ΑΡΡΕΝ, AUA 35+2 weeks, EFW ...",2024-10-01 20:38:44,stelios77,2024-10-31 19:11:07,stelios77,1
9,4236,569,2024-11-07,,0,36+2(wbd),0.0,0.0,0.0,0.0,...,,,0.0,0.0,"-υπέρηχος κυήσεως: ΑΡΡΕΝ, EFW 2900gr όλα καλά\...",2024-10-31 19:11:29,stelios77,2024-11-07 18:48:11,stelios77,1


In [6]:
schema

{'admin': ['adminID',
  'Username',
  'Password',
  'Hash',
  'FirstName',
  'LastName',
  'Created',
  'CreatedBy',
  'LastUpdated',
  'LastUpdatedBy',
  'IsActive',
  'isAdmin'],
 's_org_ext': ['s_org_extID',
  'firstName',
  'lastName',
  'fatherName',
  'dateOfBirth',
  'maritalStatus',
  'nationality',
  'occupation',
  'streetName',
  'streetNumber',
  'city',
  'postalCode',
  'county',
  'homePhone',
  'mobilePhone',
  'alternativePhone',
  'email',
  'insurance',
  'insuranceComment',
  'amka',
  'spouseName',
  'spouseDateOfBirth',
  'spouseOccupation',
  'Created',
  'CreatedBy',
  'LastUpdated',
  'LastUpdatedBy',
  'IsActive'],
 'tblappointment': ['tblAppointmentID',
  'AppointmentDate',
  'AppointmentStartTime',
  'AppointmentDuration',
  's_org_extID',
  'AppointmentComments',
  'IsActive',
  'AppointmentEndTime',
  'AppointmentType',
  'Created',
  'CreatedBy',
  'LastUpdated',
  'LastUpdatedBy'],
 'tblbirthtype': ['BirthTypeID', 'BirthTypeValue'],
 'tblgynhistory': ['G